# 创建自己的图结构

In [101]:
from typing import List, Mapping, Any, Set


class Graph:
    def __init__(self):
        # key 是该节点的唯一标识。 key 等于待转换的图结构中节点的表示方式, 比如其他图中节点的表示方式是 1,2,3, 或者 'A', 'B', 'C', 那么 key 就是对应的值
        # value 是具体的节点。 value 就是我们利用待转换的图结构中的节点, 所创建出来的属于我们自己结构的节点。
        self.nodes: Mapping[Any, Node] = {}
        self.edges: Set[Edge] = set()


class Node:
    def __init__(self, val):
        self.val = val
        self.ind = 0
        self.outd = 0
        self.nexts: List[Node] = []
        self.edges: List[Edge] = []


class Edge:
    def __init__(self, fr: Node, to: Node, weight=0):
        self.fr = fr
        self.to = to
        self.weight = weight


In [102]:
""" 支持的结构:
matrix = [
    [权值, fr节点, to节点], # 单向边: 
]
"""
def generate_graph2(matrix: List[List[int]]):
    graph = Graph()
    for item in matrix: # 每一个 item 都是一条边
        weight, fr, to = item # 
        # 判断边的两个节点是否已存在, 不存在则新增
        if fr not in graph.nodes:
            graph.nodes[fr] = Node(fr)
        if to not in graph.nodes:
            graph.nodes[to] = Node(to)
        # 不管节点存不存在, 新的边肯定是要添加的, 节点的属性也是要更新的
        fr_node = graph.nodes.get(fr)
        to_node = graph.nodes.get(to)
        new_edge = Edge(fr_node, to_node, weight)
        graph.edges.add(new_edge) # 新的边
        fr_node.edges.append(new_edge) # fr 增加一个邻接边
        fr_node.nexts.append(to_node) # fr 增加一个邻接点
        fr_node.outd += 1 # fr 的出度 + 1
        to_node.ind += 1  # to 的入度 + 1
    return graph

In [103]:
""" 支持的结构:
matrix = [
    [节点, 节点], # 双向边
]
"""
def generate_graph(matrix):
    graph = Graph()
    for one_edge in matrix:
        n1, n2 = one_edge # 双向边
        # 添加新节点
        if n1 not in graph.nodes:
            graph.nodes[n1] = Node(n1)
        if n2 not in graph.nodes:
            graph.nodes[n2] = Node(n2)
        n1_node, n2_node = graph.nodes.get(n1), graph.nodes.get(n2)
        e1, e2 = Edge(n1_node, n2_node), Edge(n2_node, n1_node)
        # 添加边
        graph.edges.add(e1)
        graph.edges.add(e2)
        n1_node.edges.append(e1)
        n2_node.edges.append(e2)
        # 更新节点属性
        n1_node.nexts.append(n2_node)
        n2_node.nexts.append(n1_node)
        n1_node.outd += 1
        n1_node.ind += 1
        n2_node.outd += 1
        n2_node.ind += 1
    return graph

# TEST - 广度优先遍历 BFS

In [104]:
from queue import Queue


# 广度有限遍历, 先处理好当前节点的所有邻接点, 再去处理邻接点的邻接点。 已经处理过的节点不要再处理。
# 利用 queue 保存待处理的节点, 利用 set 保存已经处理过的节点
def bfs(node):
    if node is None:
        return
    queue = Queue()
    selected_node = set()
    queue.put(node)
    selected_node.add(node)
    while not queue.empty():
        n = queue.get()
        print(n.val, end=' ')  # 出队列时处理
        # 处理完后, 先处理该节点的所有邻接点, 所以将他们先全部添加到队列中
        for wait_node in n.nexts:
            # 处理过的节点不要再处理
            if wait_node not in selected_node:
                selected_node.add(wait_node)
                queue.put(wait_node)
    print('')


In [105]:
matrix = [
    ['A', 'B'],
    ['A', 'C'],
    ['A', 'D'],
    ['B', 'C'],
    ['D', 'C'],
    ['E', 'C'],
    ['D', 'F'],
]
graph = generate_graph(matrix)
bfs(graph.nodes.get('E'))

E C A B D F 


# TEST - 深度优先遍历 DFS

In [106]:
# 深度优先, 一直逮着一个往下处理, 没得处理了才回退(出栈)
def dfs(node):
    stack = []
    selected_node = set()
    stack.append(node)
    print(node.val, end=' ')  # 处理时刻
    selected_node.add(node)  # 处理后标记起来

    while 0 != len(stack):  # stack 为空说明都被处理过了
        n = stack.pop()  # 将处理过的拿走。
        # 逮着 n 继续往下处理
        for next_node in n.nexts:
            if next_node not in selected_node:  # 逮着一个没处理过的
                stack.append(n)  # 此时还无法确定 n 的 nexts 都处理过了, 所以重新将 n 压栈
                stack.append(next_node)  # 拿到 stack 等待处理
                print(next_node.val, end=' ')  # 处理时刻
                selected_node.add(next_node)  # 处理后标记
                break  # 逮着一个就马上处理, 这才是 "深度优先"
    print('')

In [107]:
matrix = [
    ['A', 'B'],
    ['A', 'C'],
    ['B', 'D'],
    ['B', 'E'],
]
graph = generate_graph(matrix)
dfs(graph.nodes.get('A'))

A B D E C 


# [拓扑排序](https://www.nowcoder.com/practice/88f7e156ca7d43a1a535f619cd3f495c)

In [108]:
import sys


def get_input():
    node_num, _ = sys.stdin.readline().split()
    node_num = int(node_num)

    # 下标就是节点, 值就是该节点的入度。
    nodes_ind = [0] * node_num
    # 下标就是节点, 值是该节点的邻接点集合。
    nodes_nexts = [[] for _ in range(node_num)]

    for line in sys.stdin:
        fr, to = line.split()
        fr, to = int(fr)-1, int(to)-1  # 输入的节点是从 1 开始
        nodes_ind[to] += 1
        nodes_nexts[fr].append(to)

    return nodes_ind, nodes_nexts


def topology(nodes_ind, nodes_nexts):

    zero_nodes = []  # 存储入度为 0 的节点
    result = []  # 存储拓扑排序后的节点

    # 初始化 zero_nodes
    for node, ind in enumerate(nodes_ind):
        if ind == 0:
            zero_nodes.append(node)

    if len(zero_nodes) == 0:
        return []

    while len(zero_nodes) != 0:
        node = zero_nodes.pop()
        result.append(node)
        for next_node in nodes_nexts[node]:
            nodes_ind[next_node] -= 1   # 消除该节点的入度影响
            if nodes_ind[next_node] == 0:   # 每次消除都应该会有新的节点入度为 0
                zero_nodes.append(next_node)

    return result if len(result) == len(nodes_ind) else []


def main():# if __name__ == "__main__":

    nodes_ind, nodes_nexts = get_input()
    result = topology(nodes_ind, nodes_nexts)

    print(
        ' '.join(str(node+1) for node in result)
        if 0 != len(result) else 
        '-1'
    )


# TEST - 拓扑排序

In [109]:
class Graph:
    def __init__(self):
        self.nodes = {}
class Node:
    def __init__(self, val):
        self.val = val
        self.ind = 0 # 入度
        self.nexts = [] # 邻接点 集合

In [110]:
def generate_graph(matrix):
    graph = Graph()
    for edge in matrix:
        fr, to = edge
        if fr not in graph.nodes:
            graph.nodes[fr] = Node(fr)
        if to not in graph.nodes:
            graph.nodes[to] = Node(to)
        fr_node, to_node = graph.nodes.get(fr), graph.nodes.get(to)
        fr_node.nexts.append(to_node)
        to_node.ind += 1
    return graph

In [111]:
def topology(graph):
    ind_map = {} # 存储各节点的入度
    zero_nodes = [] # 存储入度为 0 的节点
    result = [] # 存储排序后的节点值
    # 初始化 ind_map 和 zero_nodes
    for node in graph.nodes.values():
        ind_map[node] = node.ind
        if node.ind == 0:
            zero_nodes.append(node)
    if len(zero_nodes) == 0:
        return []
    while len(zero_nodes) != 0:
        node = zero_nodes.pop()
        result.append(node)
        for next_node in node.nexts:
            ind_map[next_node] -= 1  # 更新邻接点的入度
            if ind_map[next_node] == 0: 
                zero_nodes.append(next_node) # 更新 zero_nodes
    
    return result if len(result) == len(ind_map) else []

In [113]:
matrix = [
    ['F', 'E'],
    ['F', 'D'],
    ['E', 'B'],
    ['D', 'B'],
    ['C', 'A'],
    ['C', 'B'],
    ['B', 'A'],
]
graph =generate_graph(matrix)
for node in topology(graph):
    print(node.val, end=' ')

C F D E B A 

# [最小生成树 MST]()

In [ ]:
""" 
kruskal 算法, 核心在于 edge 的 fr, to 和 w 权值。 而节点只需要有个 val 就行。
算法步骤就是, 先将所有变存储在有序表中, 然后每次取出最小边, 判断该边是否导致成环。
判断成环的方法就是, 查看该边的两个端点是否是同一个集合, 如果不是, 则加入该边, 同时将两个端点的集合设置为同一个。

由于题意只需要最低成本, 所以 "加入该边" 就是 "加上该边的权值(成本)" 
而每户人家都是用数字表示, 所以可以利用数组下标作为每户的标识, 数组的值就是对应的集合。
初始时每户人家集合都不同, 所以可以直接将值设置为下标值。
当合并集合时, 就将两个元素的值修改为同一个**地址**即可。
    不过, 即使这样只不过是将哈希表的常数时间优化为数组的常数时间罢了, 时间复杂度还是很高的

各路的权值是不变的, 所以存储边不一定得用有序表, 只要保证有序即可。
"""
class UnionFind:
    def __init__(self, house_num):
        # connected_house 的下标就是每户人家, 值为该户人家所在集合中的所有户人家, 初始时, 每个集合内只会自己, 并且不同户的集合不同(创建出的数组地址不同)
        self.connected_house = [[house] for house in range(house_num+1)] # 因为每户编号从 1 开始, 所以要加 1

    def is_same_set(self, house1, house2):
        return self.connected_house[house1] == self.connected_house[house2]

    def union(self, house1, house2):
        # 将两个集合合并时要注意, 不是简单的合并两户人家, 而是要确保两户人家所在集合中的每户人家都属于同一个集合
        
        house1_set = self.connected_house[house1]
        for house in self.connected_house[house2]:
            house1_set.append(house) # 先将 house2 所在集合的所有户人家添加到 house1 集合中
            self.connected_house[house] = house1_set # 同时将这些人家的集合重定位到 house1 集合中, 这样以后比较时才会认为这些集合内的人家是同一个集合。



def kruskal(house_num, road_cost):
    union_find = UnionFind(house_num) # 并查集
    tco = 0 # 总成本
    # 对 road_cost 排序, 按成本从高到低排序, 因为我们会先取出最后一个。 
    ordered_road_cost = sorted(road_cost, key=lambda x: 0-x[2]) # road_cost 中每个元素为 [house1, house2, 修路成本]
    while 0 != len(ordered_road_cost):
        house1, house2, cost = ordered_road_cost.pop() # 每次取出最小的
        if not union_find.is_same_set(house1, house2): # 判断路两端的两户人家是否连通
            tco += cost # 不连通, 则修建这条路
            union_find.union(house1, house2) # 同时将这两户人家所在的两个集合连通。
    return tco

# TEST - 最小生成树 MST

## Kruskal

In [180]:
class Graph:
    def __init__(self):
        self.nodes = {}
        self.edges = set()
class Node:
    def __init__(self, val):
        self.val = val 
class Edge:
    def __init__(self, fr, to, w):
        self.fr = fr 
        self.to = to 
        self.w = w # 权值 
    def __lt__(self, other):
        return self.w < other.w

In [181]:
# [n1, n2, w] 双向边
def generate_graph(matrix):
    graph = Graph()
    for edge in matrix:
        n1, n2, w = edge 
        if n1 not in graph.nodes:
            graph.nodes[n1] = Node(n1)
        if n2 not in graph.nodes:
            graph.nodes[n2] = Node(n2)
        # 获取节点
        n1_node, n2_node = graph.nodes.get(n1), graph.nodes.get(n2)
        # 创建边, 两条!
        e1, e2 = Edge(n1_node, n2_node, w), Edge(n2_node, n1_node, w)
        # 添加边, 两个节点都要!
        graph.edges.add(e1); graph.edges.add(e2)
    return graph


In [182]:
class SimpleUnionFind:
    def __init__(self, nodes):
        # key 是某个集合中的一个代表性节点, 随意
        # value 是节点集合, 表示这些节点都是同一个集合
        self.set_map = {}
        # 为每一个节点都创建一个集合, 此时每个节点都不连通
        for node in nodes:
            _set = set()
            _set.add(node)
            self.set_map[node] = _set

    def is_same_set(self, fr, to):
        fr_set = self.set_map.get(fr)
        to_set = self.set_map.get(to)
        return fr_set == to_set

    def union(self, fr, to):
        fr_set = self.set_map.get(fr)
        to_set = self.set_map.get(to)
        for to_node in to_set:
            fr_set.add(to_node)
            # 以后访问 to_set 中的每一个节点 to_node 时, 都是发现 to_node 和 fr_set 中的节点相同, 即属于同一个集合
            self.set_map[to_node] = fr_set


In [183]:
from queue import PriorityQueue

def kruskal(graph):
    # 为图中每一个节点都生成属于自己的集合
    union_find = SimpleUnionFind(graph.nodes.values())
    # 优先级队列, 按权值从小到大顺序存储图的边
    priority_queue = PriorityQueue()
    for edge in graph.edges:
        priority_queue.put(edge)
    mst_edges = set() # 生成树
    while not priority_queue.empty():
        # 每次取出最小边
        edge = priority_queue.get()
        # 判断该边是否会导致生成树成环
        if not union_find.is_same_set(edge.fr, edge.to):
            # 不成环, 可以添加该边。
            mst_edges.add(edge)
            # 加入该边后, 要将两段节点合并为一个集合
            union_find.union(edge.fr, edge.to)
    return mst_edges 

In [184]:
road_cost = [[5,3,8],[1,3,6],[2,5,4],[2,3,5],[4,5,6],[3,4,3],[2,4,8],[1,2,2],[1,4,5],[5,6,2]]
graph = generate_graph(road_cost)
tco = 0
for edge in kruskal(graph):
    tco += edge.w 
print(tco)

16


## Prim 

In [174]:
class Graph:
    def __init__(self):
        self.nodes = {}
class Node:
    def __init__(self, val):
        self.val = val # 值还是保存着, 虽然没什么用。
        self.edges = []
class Edge:
    def __init__(self, to, w): # Prim 算法不在乎这条边的 from 点, 只在乎 to 点
        self.to = to 
        self.w = w # 权值 
    def __lt__(self, other):
        return self.w < other.w

In [175]:
# [n1, n2, w] 双向边
def generate_graph(matrix):
    graph = Graph()
    for edge in matrix:
        n1, n2, w = edge 
        if n1 not in graph.nodes:
            graph.nodes[n1] = Node(n1)
        if n2 not in graph.nodes:
            graph.nodes[n2] = Node(n2)
        # 获取节点
        n1_node, n2_node = graph.nodes.get(n1), graph.nodes.get(n2)
        # 创建边, 两条!
        e1, e2 = Edge(n1_node, w), Edge(n2_node, w)
        # 添加边, n1 要有指向 n2 的边, n2 要有指向 n1 的边
        n1_node.edges.append(e2); n2_node.edges.append(e1)
    return graph


In [178]:
from queue import PriorityQueue

def prim(graph):
    pq = PriorityQueue() # 存储被解锁(可供选择)的边, 可以按从小到大次序取出
    mst_nodes_set = set() # 存储 MST 节点
    mst_edges_set = set() # 存储 MST 边

    for node in graph.nodes.values(): # 这个 for 循环是处理 graph 中存在多片不连通的区域的情况
        if node not in mst_nodes_set:

            mst_nodes_set.add(node) # 随机添加一个点
            for edge in node.edges:
                pq.put(edge)  # 添加一个点后, 将会解锁一些边
            
            # 以该边为起始, 依次将所有可连通的节点依次添加进 MST
            while not pq.empty():
                edge = pq.get() # 每次取出一条权值最小的边
                to_node = edge.to
                if to_node not in mst_nodes_set: # 要求该边的 to 端不是 mst 内部的节点
                    mst_nodes_set.add(to_node)
                    mst_edges_set.add(edge)
                    # 新增一个点, 解锁一些边。 虽然可能会加入一些已经被选取的边, 但不影响结果。 只是常数时间稍微慢了点
                    for new_edge in to_node.edges:
                        pq.put(new_edge)
    return mst_edges_set

In [179]:
road_cost = [[5,3,8],[1,3,6],[2,5,4],[2,3,5],[4,5,6],[3,4,3],[2,4,8],[1,2,2],[1,4,5],[5,6,2]]
graph = generate_graph(road_cost)
tco = 0
for edge in prim(graph):
    tco += edge.w 
print(tco)

16


# [Dijkstra 算法最短路径](https://www.nowcoder.com/practice/7c1740c3d4ba4b3486df4847ee6e8fc7)

In [ ]:
import sys
import heapq
input = sys.stdin.readline


def run(start=1):

    find, m = map(int, input().split())
    node_nexts = [[] for _ in range(5001)] # 存储每个节点的邻接点 nexts 和到邻接点的距离 
    for _ in range(m):  # 输出的条数有大于 m 的情况
        n1, n2, w = map(int, input().split())
        node_nexts[n1].append((n2, w))
        node_nexts[n2].append((n1, w))

    """ dijkstra """
    dist_table = [None] * 5001 # 节点数最多是 5000, 并且节点编号是递增的
    dist_table[start] = 0
    wait_table = [[0, 1]]  # [邻接点距离, 邻接点], 因为要排序, 所以将距离放在首位。
    while len(wait_table) != 0:
        to_dist, to = heapq.heappop(wait_table)

        if to == find:
            print(to_dist)
            return

        for next_node, next_dist in node_nexts[to]:
            old_dist = dist_table[next_node]
            new_dist = to_dist + next_dist
            if old_dist is None or new_dist < old_dist:
                dist_table[next_node] = new_dist
                heapq.heappush(wait_table, [new_dist, next_node])
    print(-1)


# TEST - Dijkstra 算法 - 最短路径

In [195]:
class Graph:
    def __init__(self):
        self.nodes = {}
        self.edges = set()
class Node:
    def __init__(self, val):
        self.val = val 
        self.edges = []
class Edge:
    def __init__(self, to, w):
        self.to = to
        self.w = w

In [201]:
def generate_graph(matrix):
    graph = Graph()
    for n1, n2, w in matrix: # 无向边
        if n1 not in graph.nodes:
            graph.nodes[n1] = Node(n1)
        if n2 not in graph.nodes:
            graph.nodes[n2] = Node(n2)
        n1_node, n2_node = graph.nodes.get(n1), graph.nodes.get(n2)
        e1, e2 = Edge(n1_node, w), Edge(n2_node, w)
        graph.edges.add(e1); graph.edges.add(e2)
        n1_node.edges.append(e2); n2_node.edges.append(e1)
    return graph

In [204]:
def dijkstra(start):
    distance_map = {}  # 存储 start 节点到其他所有节点的最短距离, 没有该节点则表示距离是无穷大
    distance_map[start] = 0  # 到自己的距离是 0
    lock_node_set = set()  # 因为距离没有负值, 所以已确定的最短距离是不会再改变的

    shorted_node = get_shorted_but_unlock_node(distance_map, lock_node_set)

    while shorted_node is not None:  # 当 distance_map 中的每个节点都被锁定时, shorted_node 将会是 None
        # 每次选出的最短距离节点, 都代表该节点的距离不会再变短了, 所以可以锁定。
        # 但在锁定前, 要利用这个最短距离, 计算出下一个最短距离, 即更新其他节点的最短距离
        distance = distance_map[shorted_node]

        for edge in shorted_node.edges:
            old_distance = distance_map.get(edge.to)
            new_distance = distance + edge.w  # 利用 shorted_node 为跳板所计算出的新距离
            # 如果边的 to 端节点不在 distance_map 中, 表示 to 节点的距离是无穷, 所以无需与旧值比较, 直接更新
            if old_distance is None or new_distance < old_distance:
                distance_map[edge.to] = new_distance
        # 利用完这个最短距离节点后, 就将其锁定, 然后获取下一个最短距离节点
        lock_node_set.add(shorted_node)
        shorted_node = get_shorted_but_unlock_node(distance_map, lock_node_set)
    return distance_map


def get_shorted_but_unlock_node(distance_map, lock_node_set):
    shorted_node = None
    for node in distance_map:
        if (node not in lock_node_set) and (shorted_node is None or distance_map[node] < distance_map[shorted_node]):
            shorted_node = node
    return shorted_node


In [215]:
matrix = [
['1','2',3],
['2','4',7],
['3','4',5],
['3','1',3],
]
graph =generate_graph(matrix)
distance_map = dijkstra(graph.nodes.get('1'))
print(distance_map.get(graph.nodes.get('4')))

8


In [212]:
matrix = [
    ['A', 'B', 3],
    ['A', 'C', 15],
    ['A', 'D', 9],
    ['B', 'C', 2],
    ['D', 'C', 7],
    ['D', 'E', 16],
    ['C', 'E', 14],
    ['B', 'E', 200],
]
graph = generate_graph(matrix)
distance_map = dijkstra(graph.nodes.get('A'))
for to in distance_map:
    print(to.val, distance_map.get(to))

A 0
B 3
C 5
D 9
E 19
